In [21]:
### Cell 1: Import Libraries and Load Training Data
import pandas as pd
import os
import cv2
import numpy as np
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.decomposition import PCA
from keras.preprocessing.image import ImageDataGenerator

# Function to load and preprocess training data
def load_training_data(data_folder, target_size=(224, 224)):
    images = []
    labels = []
    label_map = {}
    current_label = 0
    data_gen = ImageDataGenerator(
        rotation_range=30,
        width_shift_range=0.3,
        height_shift_range=0.3,
        shear_range=0.3,
        zoom_range=0.3,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    for person_name in os.listdir(data_folder):
        person_folder = os.path.join(data_folder, person_name)
        if os.path.isdir(person_folder):
            label_map[current_label] = person_name
            for filename in os.listdir(person_folder):
                img_path = os.path.join(person_folder, filename)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                if img is not None:
                    img_resized = cv2.resize(img, target_size)
                    img_rgb = cv2.cvtColor(img_resized, cv2.COLOR_GRAY2RGB)
                    augmented_imgs = [img_rgb] + [
                        data_gen.random_transform(img_rgb) for _ in range(5)
                    ]
                    images.extend(augmented_imgs)
                    labels.extend([current_label] * len(augmented_imgs))
            current_label += 1

    return np.array(images), np.array(labels), label_map

# Load training data
data_folder = "../data/faces3"
images, labels, label_map = load_training_data(data_folder)

# Flatten images for PCA
def extract_eigenfaces(images, num_components=50):
    flattened_images = images.reshape(len(images), -1)
    pca = PCA(n_components=num_components, whiten=True, random_state=42)
    eigenfaces = pca.fit_transform(flattened_images)
    return eigenfaces, pca

print(f"Loaded {len(images)} images for training.")
eigenfaces, pca = extract_eigenfaces(images)

Loaded 3666 images for training.


In [23]:
### Cell 2: Create and Train the Enhanced Model
def create_cnn_with_pca_model(num_classes, input_shape=(224, 224, 3), num_components=50):
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)
    for layer in base_model.layers[:100]:
        layer.trainable = False
    
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(num_components, activation='linear', name="pca_layer")(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.3)(x)
    predictions = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Create and train the model
num_classes = len(label_map)
model = create_cnn_with_pca_model(num_classes)

print("Training the enhanced CNN model with PCA...")
history = model.fit(images, labels, batch_size=32, epochs=10, validation_split=0.2)

# Save the trained model
model_path = "../data/cnn_pca_model.h5"
model.save(model_path)
print(f"Model saved to {model_path}")

Training the enhanced CNN model with PCA...
Epoch 1/10
92/92 [==============================] - 47s 476ms/step - loss: 2.2173 - accuracy: 0.2599 - val_loss: 3.7473 - val_accuracy: 0.0000e+00
Epoch 2/10
92/92 [==============================] - 42s 455ms/step - loss: 1.2204 - accuracy: 0.6248 - val_loss: 5.0664 - val_accuracy: 0.0014
Epoch 3/10
92/92 [==============================] - 41s 451ms/step - loss: 0.5489 - accuracy: 0.8431 - val_loss: 6.5665 - val_accuracy: 0.0000e+00
Epoch 4/10
92/92 [==============================] - 43s 465ms/step - loss: 0.2265 - accuracy: 0.9461 - val_loss: 6.2652 - val_accuracy: 0.0054
Epoch 5/10
92/92 [==============================] - 42s 455ms/step - loss: 0.0906 - accuracy: 0.9823 - val_loss: 7.0339 - val_accuracy: 0.0082
Epoch 6/10
92/92 [==============================] - 43s 471ms/step - loss: 0.0471 - accuracy: 0.9908 - val_loss: 7.7132 - val_accuracy: 0.0068
Epoch 7/10
92/92 [==============================] - 45s 485ms/step - loss: 0.0237 - accura

/home/neyon/anaconda3/lib/python3.9/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model saved to ../data/cnn_pca_model.h5


In [24]:
### Cell 3: Load Model and Predict on Test Data
from tensorflow.keras.models import load_model

def predict_faces_on_folder_with_pca(model_path, test_faces_folder, label_map, target_size=(224, 224)):
    model = load_model(model_path)

    results = []

    for filename in os.listdir(test_faces_folder):
        test_image_path = os.path.join(test_faces_folder, filename)

        test_img = cv2.imread(test_image_path)
        if test_img is None:
            print(f"Failed to read {test_image_path}, skipping.")
            continue

        test_img_resized = cv2.resize(test_img, target_size)
        test_img_preprocessed = np.expand_dims(test_img_resized, axis=0) / 255.0

        predictions = model.predict(test_img_preprocessed)
        label_idx = np.argmax(predictions)
        confidence = predictions[0][label_idx]
        person_name = label_map[label_idx]

        results.append({
            "filename": filename,
            "predicted_label": person_name,
            "confidence": confidence
        })

    return results

# Predict on test data
model_path = "../data/cnn_pca_model.h5"
test_faces_folder = "../data/faces3_test"
results = predict_faces_on_folder_with_pca(model_path, test_faces_folder, label_map)

1/1 [==============================] - 0s 27ms/step


In [29]:
### Cell 4: Group Results by Image
def group_results_by_image(results, confidence_threshold=0):
    grouped_data = {}
    for result in results:
        base_filename = result["filename"].split("_face")[0]
        if result["confidence"] < confidence_threshold:
            continue
        if base_filename not in grouped_data:
            grouped_data[base_filename] = []
        grouped_data[base_filename].append(result["predicted_label"].lower())

    grouped_results = [{"filename": filename, "predicted_labels": labels} for filename, labels in grouped_data.items()]
    return grouped_results

grouped_results = group_results_by_image(results)

In [30]:
### Cell 5: Create Submission File
def create_submission_csv_from_grouped_results(grouped_results, output_csv_path, test_images_folder):
    all_filenames = sorted(
        [os.path.splitext(filename)[0] for filename in os.listdir(test_images_folder) 
         if filename.endswith((".jpg", ".jpeg", ".png"))]
    )

    grouped_dict = {group["filename"].split(".")[0]: ";".join(group["predicted_labels"]) for group in grouped_results}

    submission_data = []
    for filename in all_filenames:
        label_name = grouped_dict.get(filename, "nothing")
        submission_data.append({"image": filename, "label_name": label_name})

    submission_df = pd.DataFrame(submission_data)
    submission_df.to_csv(output_csv_path, index=False)
    print(f"Submission file saved to {output_csv_path}")

# Example usage
test_images_folder = "../data/images/test_images/cleaned_images"
output_csv_path = "../submission.csv"
create_submission_csv_from_grouped_results(grouped_results, output_csv_path, test_images_folder)

Submission file saved to ../submission.csv
